# Tensorflow Basics

In [3]:
import numpy as np
import os

%matplotlib
import matplotlib
import matplotlib.pyplot as plot
import time

Using matplotlib backend: MacOSX


## Create and Running a Graph

In [4]:
import tensorflow as tf

def reset_graph(seed=42):
    tf.reset_default_graph()  # reset default graph
    tf.set_random_seed(seed)  # seed tensorflow
    np.random.seed(seed)      # seed numpy

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Variables

Tensorflow variables are nodes in the execution graph that holds a variable value. Value could be simple types like int, float or complex types like tensors. Various ops could be performed on variables.

**Note:** 
- Use tf.get_variable function instead of tf.Variable constructor
- Get variable returns existing variable. So remeber to reset the graph to reset values held by the variables


Overview: https://www.tensorflow.org/guide/variables

In [5]:
reset_graph()

# if get_variable is used, scope should be set to AUTO_REUSE
with tf.variable_scope("foo", reuse=tf.AUTO_REUSE):
    x = tf.get_variable('x', initializer=3)
    y = tf.get_variable('y', initializer=4)
    a = tf.constant(2, name='a')

    t1 = tf.get_variable('t1', shape=(2, 4), initializer=tf.zeros_initializer)
    t2 = tf.get_variable('t2', shape=(2, 4), initializer=tf.random_normal_initializer)

# create variable using Variable. This will not reuse existing variables
v = tf.Variable(2, name='v') # get_variable is preffered

## Graph

Tensorflow uses *default graph* for variables when not specified. Usage of explicit graph is given below

In [6]:
graph = tf.Graph()

with graph.as_default(), tf.variable_scope("bar", reuse=tf.AUTO_REUSE):
    k = tf.get_variable('k', initializer=19, dtype=tf.int32)
    
print(x.graph == tf.get_default_graph())
print(k.graph == graph)

True
True


## Operations

Tensorflow operation is performed below. Operations are same as math operations due to operator overloading

In [7]:
f = x*x*y + y + a

## Evaluating Tensor Graph

Tensorflow uses lazy evaluation and objects are not initialized at creation. 

In [8]:
# variable sare not evaluated
x, f, a

(<tf.Variable 'foo/x:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'add_1:0' shape=() dtype=int32>,
 <tf.Tensor 'foo/a:0' shape=() dtype=int32>)

### Evaluate Verbose Version

In [9]:
# initialize (verbose)

sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
# a doesn't need to be initialized since it's a constant 

print(x.eval(session=sess))   # x should be initialized to call this function
print(a.eval(session=sess))   # a doesn't need to be initialized since it's a const

result = sess.run(f)
print(result)
sess.close()

3
2
42


### Evaluate Usable Version

In [10]:
# use global initialized to initialize all the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run() # run global initializer
    result = f.eval()
    
    print(result)

42


## Lifecylce of Nodes

In [11]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x + 3

# This uses two passes through the graph, since node y and z are evaluated seperately.
# Value of x is evaluated twice since graph is reset at each run
with tf.Session() as sess:
    print(y.eval(session=sess))
    print(z.eval(session=sess))
    
print("\nEfficient way")
# using one pass evaluation is efficient since nodes are reused
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)


10
8

Efficient way
10
8


## Tensorflow Autodiff (Back-propagation)

In [12]:
reset_graph()

def my_func(w, x):
    with tf.variable_scope("0_level", reuse=tf.AUTO_REUSE):
        f_0 = tf.exp(w[0, 0] + w[0, 1]*x)
    with tf.variable_scope("1_level", reuse=tf.AUTO_REUSE):
        f_1 = tf.exp(w[1, 0] + w[1, 1]*f_0)
    with tf.variable_scope("2_level", reuse=tf.AUTO_REUSE):
        f_2 = tf.exp(w[2, 0] + w[2, 1]*f_1)
        
    return f_2, f_1, f_0

In [13]:
w_0 = np.stack((np.zeros(3), np.ones(3))).T.astype(np.float32)
w_0

array([[0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [14]:
# variables with inializers
with tf.variable_scope("parameters", reuse=tf.AUTO_REUSE):
    w_0 = tf.get_variable("w_0", initializer=w_0, dtype=tf.float32)
    x = tf.get_variable("x", initializer=1.0, dtype=tf.float32)

# define the function
f_2, f_1, f_0 = my_func(w_0, x)

# calculate gradients
grad = tf.gradients(f_2, w_0)
grad

[<tf.Tensor 'gradients/AddN:0' shape=(3, 2) dtype=float32>]

In [15]:
# node for initializer
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    # evaluate the expressions
    # remeber that f_2, f_1 and f_0 are not functions, they are nodes in the graph
    grads, f_vals = sess.run([grad, [f_2, f_1, f_0]])
    
print("Function values: ", f_vals)
print("Gradients: ", grads)

Function values:  [3814273.0, 15.154261, 2.7182817]
Gradients:  [array([[1.5712344e+08, 1.5712344e+08],
       [5.7802488e+07, 1.5712344e+08],
       [3.8142730e+06, 5.7802488e+07]], dtype=float32)]


<img src="calculate_derivative.png" width="600">

### Visualize Tensorflow Graph

In [16]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [18]:
show_graph(tf.get_default_graph())